In [106]:
import pandas as pd
import numpy as np
import warnings
from sklearn.model_selection import train_test_split
warnings.filterwarnings('ignore')
import keras
from sklearn.metrics import mean_absolute_error,mean_squared_error
import os

In [107]:
print("Start reading data frm csv")
anime = pd.read_csv('../input/anime.csv')
rating = pd.read_csv('../input/rating.csv')
print('anime.csv shape', anime.shape)
print('rating.csv shape', rating.shape)
print(anime[:1])
print(rating[:1])

Start reading data frm csv
anime.csv shape (12294, 7)
rating.csv shape (7813737, 3)
   anime_id            name   ...   rating members
0     32281  Kimi no Na wa.   ...     9.37  200630

[1 rows x 7 columns]
   user_id  anime_id  rating
0        1        20      -1


In [108]:
rating = rating[rating.rating != -1]
print('After removing -1 entries')
print('rating.shape',rating.shape)
print(rating.head())
print (len(rating.user_id.unique()), len(rating.anime_id.unique()))

After removing -1 entries
rating.shape (6337241, 3)
     user_id  anime_id  rating
47         1      8074      10
81         1     11617      10
83         1     11757      10
101        1     15451      10
153        2     11771      10
69600 9927


In [109]:
limit_user_id = 1000
limit_anime_id = int(limit_user_id*2)
rating = rating[rating.user_id<=limit_user_id ]
rating = rating[rating.anime_id<=limit_anime_id]

print('after limiting users to id<=', limit_user_id, 'animes to id<=', limit_anime_id)
print(rating.shape)
print(rating.head())

after limiting users to id<= 1000 animes to id<= 2000
(17984, 3)
     user_id  anime_id  rating
156        3        20       8
157        3       154       6
158        3       170       9
159        3       199      10
160        3       225       9


In [110]:
u_users = rating.groupby(['user_id'], as_index=False).count()
u_users = len(u_users)
unique_users = limit_user_id

u_animes = rating.groupby(['anime_id'], as_index=False).count()
u_animes = len(u_animes)
unique_animes = limit_anime_id#len(unique_animes)

print('The chunked dataset\'s unique_users',u_users, 'unique_animes',u_animes)
print('When limiting users to id<=', limit_user_id, 'animes to id<=', limit_anime_id)
unique_users += 1
unique_animes += 1

The chunked dataset's unique_users 851 unique_animes 1232
When limiting users to id<= 1000 animes to id<= 2000


In [111]:
print(train.head())

       user_id  anime_id  rating
20062      210       164       9
24451      245       463       7
52937      467       173       6
42990      394       215       7
46850      423       916       9


In [112]:
dataset = rating

In [113]:
print(dataset.head())

     user_id  anime_id  rating
156        3        20       8
157        3       154       6
158        3       170       9
159        3       199      10
160        3       225       9


In [114]:
dataset.user_id = dataset.user_id.astype('category').cat.codes.values
dataset.anime_id = dataset.anime_id.astype('category').cat.codes.values

train, test = train_test_split(dataset, test_size=0.25)

train = rating.sample(frac=0.75,random_state=5)
test = rating.drop(train.index)

print('train.shape',train.shape)
print(train.head())

print('test.shape',test.shape)
print(test.head())

y_true = test.rating

train.shape (13488, 3)
       user_id  anime_id  rating
37766      354       432      10
20116      210      1032       9
71149      636       656       7
18740      199       607       7
94224      822      1045       8
test.shape (4496, 3)
     user_id  anime_id  rating
159        0       164      10
161        0       280       6
166        0       815       7
168        0       818       7
172        0      1013       7


In [116]:
n_latent_factors_user = 8
n_latent_factors_movie = 10
n_latent_factors_mf = 3


movie_input = keras.layers.Input(shape=[1],name='Item')
movie_embedding_mlp = keras.layers.Embedding(n_movies + 1, n_latent_factors_movie, name='Movie-Embedding-MLP')(movie_input)
movie_vec_mlp = keras.layers.Flatten(name='FlattenMovies-MLP')(movie_embedding_mlp)
movie_vec_mlp = keras.layers.Dropout(0.2)(movie_vec_mlp)

movie_embedding_mf = keras.layers.Embedding(n_movies + 1, n_latent_factors_mf, name='Movie-Embedding-MF')(movie_input)
movie_vec_mf = keras.layers.Flatten(name='FlattenMovies-MF')(movie_embedding_mf)
movie_vec_mf = keras.layers.Dropout(0.2)(movie_vec_mf)


user_input = keras.layers.Input(shape=[1],name='User')
user_vec_mlp = keras.layers.Flatten(name='FlattenUsers-MLP')(keras.layers.Embedding(n_users + 1, n_latent_factors_user,name='User-Embedding-MLP')(user_input))
user_vec_mlp = keras.layers.Dropout(0.2)(user_vec_mlp)

user_vec_mf = keras.layers.Flatten(name='FlattenUsers-MF')(keras.layers.Embedding(n_users + 1, n_latent_factors_mf,name='User-Embedding-MF')(user_input))
user_vec_mf = keras.layers.Dropout(0.2)(user_vec_mf)


concat = keras.layers.concatenate([movie_vec_mlp, user_vec_mlp])
concat_dropout = keras.layers.Dropout(0.2)(concat)
dense = keras.layers.Dense(200,name='FullyConnected')(concat_dropout)
dense_batch = keras.layers.BatchNormalization(name='Batch')(dense)
dropout_1 = keras.layers.Dropout(0.2,name='Dropout-1')(dense_batch)
dense_2 = keras.layers.Dense(100,name='FullyConnected-1')(dropout_1)
dense_batch_2 = keras.layers.BatchNormalization(name='Batch-2')(dense_2)


dropout_2 = keras.layers.Dropout(0.2,name='Dropout-2')(dense_batch_2)
dense_3 = keras.layers.Dense(50,name='FullyConnected-2')(dropout_2)
dense_4 = keras.layers.Dense(20,name='FullyConnected-3', activation='relu')(dense_3)

pred_mf = keras.layers.dot([movie_vec_mf, user_vec_mf],axes=1)


pred_mlp = keras.layers.Dense(1, activation='relu',name='Activation')(dense_4)

combine_mlp_mf = keras.layers.concatenate([pred_mf, pred_mlp])
result_combine = keras.layers.Dense(100,name='Combine-MF-MLP')(combine_mlp_mf)
deep_combine = keras.layers.Dense(100,name='FullyConnected-4')(result_combine)


result = keras.layers.Dense(1,name='Prediction')(deep_combine)


model = keras.Model([user_input, movie_input], result)
opt = keras.optimizers.Adam(lr =0.01)
model.compile(optimizer='adam',loss= 'mean_absolute_error')

print (model.summary())


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Item (InputLayer)               (None, 1)            0                                            
__________________________________________________________________________________________________
User (InputLayer)               (None, 1)            0                                            
__________________________________________________________________________________________________
Movie-Embedding-MLP (Embedding) (None, 1, 10)        12330       Item[0][0]                       
__________________________________________________________________________________________________
User-Embedding-MLP (Embedding)  (None, 1, 8)         6816        User[0][0]                       
__________________________________________________________________________________________________
FlattenMov

In [117]:
import time
print('model fitting started')
start = time.time()
history = model.fit([train.user_id, train.anime_id], train.rating, epochs=25, verbose=1, validation_split=0.1)
end = time.time()
print('model fitting done in (s):', end-start)

model fitting started
Train on 12139 samples, validate on 1349 samples
Epoch 1/25
12139/12139 [==============================] - 7s 581us/step - loss: 1.6658 - val_loss: 1.2446
Epoch 2/25
12139/12139 [==============================] - 4s 302us/step - loss: 1.1598 - val_loss: 1.0949
Epoch 3/25
12139/12139 [==============================] - 4s 298us/step - loss: 1.0745 - val_loss: 1.0519
Epoch 4/25
12139/12139 [==============================] - 4s 298us/step - loss: 1.0139 - val_loss: 1.0460
Epoch 5/25
12139/12139 [==============================] - 4s 301us/step - loss: 0.9754 - val_loss: 1.0341
Epoch 6/25
12139/12139 [==============================] - 4s 311us/step - loss: 0.9472 - val_loss: 1.0741
Epoch 7/25
12139/12139 [==============================] - 4s 299us/step - loss: 0.9335 - val_loss: 1.0258
Epoch 8/25
12139/12139 [==============================] - 4s 296us/step - loss: 0.9022 - val_loss: 1.0236
Epoch 9/25
12139/12139 [==============================] - 4s 297us/step - loss: 0

In [118]:
from sklearn.metrics import mean_squared_error
y_hat_2 = np.round(model.predict([test.user_id, test.anime_id]),0)


In [119]:
y_predicted = []
for i in range(len(y_hat_2)):
    y_predicted.append(y_hat_2[i][0])
    
y_actual = y_true.values

In [121]:
m = mean_absolute_error(y_actual, y_predicted)
r = mean_squared_error(y_actual, y_predicted)

print('For DL')
print('For user_ids<',unique_users,'and anime_ids<', unique_animes)
print('The chunked dataset\'s unique_users',u_users, 'unique_animes',u_animes)
#r,m = GetErrors(cosine_ratings_predictions)
print('With train data of shape:',train.shape)
print('and test data of shape:',test.shape)
print('MAE:', m)
print('RMSE:', r)

#print(mean_absolute_error(y_true, model.predict([test.user_id, test.anime_id])))

For DL
For user_ids< 1001 and anime_ids< 2001
The chunked dataset's unique_users 851 unique_animes 1232
With train data of shape: (13488, 3)
and test data of shape: (4496, 3)
MAE: 1.0242437722419928
RMSE: 2.002446619217082
